# Building a boolean network from a knowledge graph

Use SigNOR

Given a list of genes for which we want to construct a BN:

1. Find the Steiner subgraph for the given list of genes.
2. For each node in the subgraph, find all if its input nodes (i.e. all edges leading into that node).
3. For each such node, there is an activating relation if the edge is "up-regulates", and a repressing relation if the edge is "down-regulates".
4. Combine all of the edges with a "joiner function" - AND, OR, repressor wins, etc.

Or, given an existing model (with some number of genes), a list of nodes, and a knowledge graph, how do we incorporate the new nodes into the model?


In [1]:
from BNMPy import build_bn_from_kg

In [2]:
bn_string, relations = build_bn_from_kg.load_signor_network(['TNFAIP3', 'CCND1', 'MACF1'])

['UNIPROT::P21580', 'UNIPROT::P24385', 'UNIPROT::Q9UPN3']
['UNIPROT::P21580', 'UNIPROT::P24385', 'UNIPROT::Q9UPN3']


In [3]:
print(bn_string)

ITCH = ITCH
TNFAIP3 = (ITCH)
CCND1 = (! GSK3B) & (GLI1)
GSK3B = (GSK3B) & (! MACF1)
MACF1 = (! GSK3B)
GLI1 = (! ITCH) & (GLI1)


In [4]:
bn_string, relations = build_bn_from_kg.load_signor_network(['KRAS', 'GNAS', 'TP53', 'SMAD4', 'CDKN2A', 'RNF43'], joiner='&')
print(bn_string)

['UNIPROT::P01116', 'UNIPROT::P63092', 'UNIPROT::P04637', 'UNIPROT::Q13485', 'UNIPROT::P42771', 'UNIPROT::Q68DV7']
['UNIPROT::P01116', 'UNIPROT::P63092', 'UNIPROT::P04637', 'UNIPROT::Q13485', 'UNIPROT::P42771', 'UNIPROT::Q68DV7']
GNAS = GNAS
MAPK3 = (MAPK3)
IKBKB = (! IKBKB) & (SRC)
MYC = (MAPK3) & (! CEBPA) & (! SMAD4)
CEBPA = (! MAPK3) & (CEBPA) & (! FZD2)
SRC = (GNAS) & (! SRC)
TP53 = (MAPK3)
FZD2 = (RNF43)
SMAD4 = (MAPK3)
KRAS = (SRC)
CDKN2A = (! IKBKB) & (! MYC)
RNF43 = RNF43


In [5]:
bn_string, relations = build_bn_from_kg.load_signor_network(['KRAS', 'GNAS', 'TP53', 'SMAD4', 'CDKN2A', 'RNF43'], joiner='|')
print(bn_string)

['UNIPROT::P01116', 'UNIPROT::P63092', 'UNIPROT::P04637', 'UNIPROT::Q13485', 'UNIPROT::P42771', 'UNIPROT::Q68DV7']
['UNIPROT::P01116', 'UNIPROT::P63092', 'UNIPROT::P04637', 'UNIPROT::Q13485', 'UNIPROT::P42771', 'UNIPROT::Q68DV7']
GNAS = GNAS
MAPK3 = (MAPK3)
IKBKB = (! IKBKB) | (SRC)
MYC = (MAPK3) | (! CEBPA) | (! SMAD4)
CEBPA = (! MAPK3) | (CEBPA) | (! FZD2)
SRC = (GNAS) | (! SRC)
TP53 = (MAPK3)
FZD2 = (RNF43)
SMAD4 = (MAPK3)
KRAS = (SRC)
CDKN2A = (! IKBKB) | (! MYC)
RNF43 = RNF43


In [6]:
bn_string, relations = build_bn_from_kg.load_signor_network(['KRAS', 'GNAS', 'TP53', 'SMAD4', 'CDKN2A', 'RNF43'],
                                                            joiner='inhibitor_wins')
print(bn_string)

['UNIPROT::P01116', 'UNIPROT::P63092', 'UNIPROT::P04637', 'UNIPROT::Q13485', 'UNIPROT::P42771', 'UNIPROT::Q68DV7']
['UNIPROT::P01116', 'UNIPROT::P63092', 'UNIPROT::P04637', 'UNIPROT::Q13485', 'UNIPROT::P42771', 'UNIPROT::Q68DV7']
GNAS = GNAS
MAPK3 = MAPK3
IKBKB = (!IKBKB) & (SRC)
MYC = (!CEBPA & !SMAD4) & (MAPK3)
CEBPA = (!MAPK3 & !FZD2) & (CEBPA)
SRC = (!SRC) & (GNAS)
TP53 = MAPK3
FZD2 = RNF43
SMAD4 = MAPK3
KRAS = SRC
CDKN2A = !IKBKB & !MYC
RNF43 = RNF43


In [7]:
relations

[('MAPK3', 'MAPK3', 'activate'),
 ('IKBKB', 'IKBKB', 'inhibit'),
 ('SRC', 'IKBKB', 'activate'),
 ('MAPK3', 'MYC', 'activate'),
 ('CEBPA', 'MYC', 'inhibit'),
 ('SMAD4', 'MYC', 'inhibit'),
 ('MAPK3', 'CEBPA', 'inhibit'),
 ('CEBPA', 'CEBPA', 'activate'),
 ('FZD2', 'CEBPA', 'inhibit'),
 ('GNAS', 'SRC', 'activate'),
 ('SRC', 'SRC', 'inhibit'),
 ('MAPK3', 'TP53', 'activate'),
 ('RNF43', 'FZD2', 'activate'),
 ('MAPK3', 'SMAD4', 'activate'),
 ('SRC', 'KRAS', 'activate'),
 ('IKBKB', 'CDKN2A', 'inhibit'),
 ('MYC', 'CDKN2A', 'inhibit')]

## Combining KG knowledge graphs

In [8]:
# load the Vundavalli KG
from BNMPy import booleanNetwork as bn
from BNMPy import BMatrix 

file = 'input_files/pancreatic_vundavilli_2020_fig3.txt'
network = BMatrix.load_network_from_file(file)

No initial state provided, using a random initial state


In [9]:
genes = network.nodeDict.keys()

In [10]:
bn_string, relations = build_bn_from_kg.load_signor_network(genes)

['UNIPROT::P01133', 'UNIPROT::Q99075', 'UNIPROT::P05019', 'UNIPROT::Q02297', 'UNIPROT::P60484', 'UNIPROT::Q15831', 'UNIPROT::P00533', 'UNIPROT::P04626', 'UNIPROT::P40763', 'UNIPROT::P35568', 'UNIPROT::P62993', 'UNIPROT::Q13233', 'UNIPROT::Q9Y6X8', 'UNIPROT::P45985', 'UNIPROT::Q02750', 'UNIPROT::P42336', 'UNIPROT::P45983', 'UNIPROT::O15530', 'UNIPROT::P31749', 'UNIPROT::Q13131', 'UNIPROT::P23443', 'UNIPROT::Q92934', 'UNIPROT::P24385', 'UNIPROT::P10415', 'UNIPROT::P11464']
['UNIPROT::P01133', 'UNIPROT::Q99075', 'UNIPROT::P05019', 'UNIPROT::Q02297', 'UNIPROT::P60484', 'UNIPROT::Q15831', 'UNIPROT::P00533', 'UNIPROT::P04626', 'UNIPROT::P40763', 'UNIPROT::P35568', 'UNIPROT::P62993', 'UNIPROT::Q13233', 'UNIPROT::P45985', 'UNIPROT::Q02750', 'UNIPROT::P42336', 'UNIPROT::P45983', 'UNIPROT::O15530', 'UNIPROT::P31749', 'UNIPROT::Q13131', 'UNIPROT::P23443', 'UNIPROT::Q92934', 'UNIPROT::P24385', 'UNIPROT::P10415']


In [11]:
print(bn_string)

STAT3 = (MAPK8) & (EGFR) & (MAP3K1)
CCND1 = (STAT3)
RPS6KB1 = (PDPK1)
MAPK8 = (! RPS6KB1) & (MAP2K4)
BAD = (! MAPK8) & (! AKT1)
AKT1 = (MAPK8) & (PDPK1) & (! PTEN) & (PIK3CA)
EGFR = (EGFR) & (ERBB2) & (EGF) & (HBEGF)
PDPK1 = PDPK1
PTEN = (PTEN)
ERBB2 = (EGFR) & (ERBB2) & (NRG1) & (EGF)
PIK3CA = (! PTEN) & (IRS1)
IRS1 = (! RPS6KB1) & (! MAPK8) & (! PIK3CA) & (! MAP2K1)
BCL2 = (MAPK8) & (! BAD)
MAP2K1 = (PDPK1) & (MAP2K1) & (MAP3K1)
MAP3K1 = MAP3K1
NRG1 = NRG1
STK11 = (STK11)
PPARGC1A = (! AKT1) & (PPARGC1A) & (PRKAA1)
GRB2 = (ERBB2) & (IRS1)
MAP2K4 = (! AKT1) & (MAP3K1) & (MAP2K4)
PRKAA1 = (! AKT1) & (STK11) & (! PRKAA1)
IGF1 = (PPARGC1A)
EGF = EGF
HBEGF = HBEGF


In [12]:
merged_bn_string = build_bn_from_kg.merge_network(network, bn_string)

TypeError: unsupported operand type(s) for +: 'dict_keys' and 'dict_keys'

In [ ]:
print(merged_bn_string)